In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2021-03-03 04:24:06--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  26.9MB/s    in 7.8s    

2021-03-03 04:24:14 (24.4 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [ ]:
!unzip -n ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [ ]:
import pandas as pd
df=pd.read_csv('ml-20m/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,Input,Dense,Concatenate,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import numpy as np

from sklearn.utils import shuffle
 

In [ ]:
df.userId=pd.Categorical(df.userId)
df['new_user_id']=df.userId.cat.codes

df.movieId=pd.Categorical(df.movieId)
df['new_movie_id']=df.movieId.cat.codes

df.head()

,userId,movieId,rating,timestamp,new_user_id,new_movie_id
0,1,2,3.5,1112486027,0,1
1,1,29,3.5,1112484676,0,28
2,1,32,3.5,1112484819,0,31
3,1,47,3.5,1112484727,0,46
4,1,50,3.5,1112484580,0,49


In [ ]:
new_user_id=df['new_user_id'].values
new_movie_id=df['new_movie_id'].values
ratings=df['rating'].values

In [ ]:
print(new_movie_id.shape)
print(new_user_id.shape)
print(ratings.shape)
U=len(set(new_user_id))
M=len(set(new_movie_id))
K=10

(20000263,)
(20000263,)
(20000263,)


In [ ]:
u=Input(shape=(1,))
m=Input(shape=(1,))
u_emb=Embedding(U,K)(u)
m_emb=Embedding(M,K)(m)
u_flat=Flatten()(u_emb)
m_flat=Flatten()(m_emb)
x=Concatenate()([u_flat,m_flat])
x=Dense(1024,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(1)(x)
model=Model(inputs=[u,m],outputs=x)

new_movie_id,new_user_id,ratings=shuffle(new_movie_id,new_user_id,ratings)
N=((len(new_user_id))*80)//100
movie_train=new_movie_id[:N]
user_train=new_user_id[:N]
ratings_train=ratings[:N]


movie_test=new_movie_id[N:]
user_test=new_user_id[N:]
ratings_test=ratings[N:]

#Center the output data
avg_ratings=ratings_train.mean()
ratings_train=ratings_train-avg_ratings
ratings_test=ratings_test-avg_ratings

model.compile(loss='mse',
              optimizer=SGD(lr=0.05,momentum=0.9))

r=model.fit(x=[user_train,movie_train],
            y=ratings_train,
            validation_data=([user_test,movie_test],ratings_test),
            batch_size=1024,
            epochs=50)

Epoch 1/50
15626/15626 [==============================] - 1429s 91ms/step - loss: 0.8543 - val_loss: 0.7372
Epoch 2/50
15626/15626 [==============================] - 1431s 92ms/step - loss: 0.7196 - val_loss: 0.7038
Epoch 3/50
15626/15626 [==============================] - 1428s 91ms/step - loss: 0.6901 - val_loss: 0.6979
Epoch 4/50
15626/15626 [==============================] - 1418s 91ms/step - loss: 0.6783 - val_loss: 0.6881
Epoch 5/50
15626/15626 [==============================] - 1400s 90ms/step - loss: 0.6663 - val_loss: 0.6781
Epoch 6/50
15626/15626 [==============================] - 1395s 89ms/step - loss: 0.6557 - val_loss: 0.6732
Epoch 7/50
15626/15626 [==============================] - 1392s 89ms/step - loss: 0.6485 - val_loss: 0.6745
Epoch 8/50
15626/15626 [==============================] - 1393s 89ms/step - loss: 0.6418 - val_loss: 0.6683
Epoch 9/50
15626/15626 [==============================] - 1393s 89ms/step - loss: 0.6346 - val_loss: 0.6603
Epoch 10/50
15626/15626 [===